In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import seaborn as sns

In [ ]:
path = "C:\\Users\\Marvin\\OneDrive\\Desktop\\Studium\\SS21\\Seminar\\fat_monthly.csv"
fat_monthly = pd.read_csv(path)
fat_monthly.head()

In [ ]:
fat_monthly.drop("Unnamed: 0", axis=1, inplace=True)
fat_monthly = fat_monthly.dropna(subset=["MV.USD", "RET.USD"])
fat_monthly["RM"] = 0
fat_monthly.head()

In [ ]:
fat_monthly.isna().sum()

In [ ]:
end_of_1987 = fat_monthly.loc[fat_monthly["Date"] == "1987-12-31"]
end_of_1987 = end_of_1987.sort_values(by="MV.USD", ascending=False)
end_of_1987
#Why negative return in local currency and positive currency in USD?

In [ ]:
rm = 0
mv_total = end_of_1987["MV.USD"].sum()
for index, row in end_of_1987.iterrows():
    rm += (row["MV.USD"] / mv_total) * row["RET.USD"]
    print(rm)

In [ ]:
end_of_1987["RM"] = rm
end_of_1987

In [ ]:
#Calculate the Market Return for every Date in our dataset
for date in fat_monthly["Date"].unique():
    rm = 0
    mv_total = np.array(fat_monthly.loc[fat_monthly["Date"] == date, ["MV.USD"]].sum())
    mv_comps = np.array(fat_monthly.loc[fat_monthly["Date"] == date, ["MV.USD"]])
    rets = np.array(fat_monthly.loc[fat_monthly["Date"] == date, ["RET.USD"]])
    rm = sum((mv_comps / mv_total) * rets)
    fat_monthly.loc[fat_monthly["Date"] == date, ["RM"]] = rm
    print(fat_monthly.loc[fat_monthly["Date"] == date])

In [ ]:
fat_monthly.to_csv("fat_monthly_w_rm.csv")

In [ ]:
factor_path = "C:\\Users\\Marvin\\OneDrive\\Desktop\\Studium\\SS21\\Seminar\\F-F_Research_Data_Factors.csv"
ff_factors = pd.read_csv(factor_path, skiprows=3)
ff_factors = ff_factors.rename({"Unnamed: 0" : "Date"}, axis=1)
ff_factors.head()

In [ ]:
ff_factors["Date"] = pd.to_datetime(ff_factors["Date"], format="%Y%m")
fat_monthly["Date"] = pd.to_datetime(fat_monthly["Date"], format="%Y-%m-%d")
ff_factors["ym"] = ff_factors["Date"].dt.strftime("%b %Y")
fat_monthly["ym"] = fat_monthly["Date"].dt.strftime("%b %Y")

ff_factors.drop(["Date","Mkt-RF", "SMB", "HML"], axis=1, inplace=True)

fat_monthly = fat_monthly.merge(ff_factors, on="ym")
fat_monthly

In [ ]:
fat_monthly["RMRF"] = fat_monthly["RM"] - fat_monthly["RF"]
fat_monthly["Beta"] = 0

In [ ]:
len(fat_monthly["Id"].unique())

In [ ]:
single_comp = fat_monthly.loc[fat_monthly["Id"] == "13039P"]
single_comp

In [ ]:
x = single_comp["RM"]
y = single_comp["RET.USD"]
plt.scatter(x, y)
plt.show()

In [ ]:
sns.regplot(x, y)

In [ ]:
def time_series_regression(df, factors):
    """Calculates the market betas with time-series regression"""
    X = df[factors]
    y = df["RET.USD"]
    X2 = sm.add_constant(X)
    ols  = sm.OLS(y, X2)
    mod = ols.fit()
    return mod

In [ ]:
mod = time_series_regression(single_comp, "RM")
mod.params

In [ ]:
# Market beta for every stock in our sample
#If else statement because there is one stock with only one entry that doesnt have an intercept
count = 0
for stock in fat_monthly["Id"].unique():
    temp = fat_monthly.loc[fat_monthly["Id"] == stock]
    mod = time_series_regression(temp, "RM")
    if len(mod.params == 1):
        fat_monthly.loc[fat_monthly["Id"] == stock, ["Beta"]] = mod.params[0]
    else:
        fat_monthly.loc[fat_monthly["Id"] == stock, ["Beta"]] = mod.params[1]
    count += 1
    print(count)

In [ ]:
fat_monthly.to_csv("fat_monthly_beta.csv")

In [ ]:
fat_yearly = pd.read_csv("C:\\Users\\Marvin\\OneDrive\\Desktop\\Studium\\SS21\\Seminar\\fat_yearly.csv")
fat_yearly.drop("Unnamed: 0", axis=1, inplace=True)
fat_yearly

In [ ]:
fat_monthly.loc[fat_monthly["Id"] == "13039P"].sort_values("Date")

In [ ]:
#Rank the Stocks into quintiles
test = fat_monthly.loc[fat_monthly["Date"] == "1979-12-31"]
np.ceil(test["Beta"].rank(pct=True).mul(5)).sort_values()

In [ ]:
fat_yearly.isna().sum()